### Imports

In [1]:
import torch
import torch_geometric.nn as pyg_nn
import torch.nn.functional as F

### Below we specify arbitrary constants or hiperparams

`TEST_TRAIN_SPLIT` - test size (0.2 == 20%) <br>
`ROWS_PER_EXAMPLE` - number of rows (key presses) per data chunk <br>
`NUM_HIDDEN_DIMS` - TODO <br>
`NUM_EPOCHS` - number of training epochs

In [2]:
TEST_TRAIN_SPLIT = 0.2
ROWS_PER_EXAMPLE = 150
NUM_HIDDEN_DIMS = 64
NUM_EPOCHS = 1000

file_names = ["../datasets/training/user1.tsv", "../datasets/training/user2.tsv", "../datasets/training/user3.tsv", "../datasets/training/user4.tsv"]

In [3]:
# Define a simple GCN model
class LetterGNN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_dim, num_classes, num_layers=2):
        super(LetterGNN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(pyg_nn.GCNConv(num_node_features, hidden_dim))
        for _ in range(num_layers - 1):
            self.convs.append(pyg_nn.GCNConv(hidden_dim, hidden_dim))

        self.fc = torch.nn.Linear(hidden_dim, num_classes)

    def forward(self, x, edge_index, batch):
        for conv in self.convs:
            x = x.float()
            edge_index = edge_index.long()
            x = conv.forward(x, edge_index)
            x = F.relu(x)

        # idk maybe use a different pool method
        x = pyg_nn.global_mean_pool(x, batch)

        # classify
        x = self.fc(x)

        return x

Take a look at `dataset.x size`. Size (13, 5) means there is 13 different letters in data chunk, 5 attributes for each letter.

In [4]:
from torch_geometric.data import InMemoryDataset
from sklearn.model_selection import train_test_split
from utils import data_loader

mode = data_loader.LoadMode.DROP
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# This is a list of lists of Data objects - datasets
# each dataset comes from a different input file - a different user
list_of_datasets = [
    data_loader.load_from_file(filename, mode=mode, y=torch.tensor([i]), rows_per_example=ROWS_PER_EXAMPLE)
    for i, filename in enumerate(file_names)]

num_features = list_of_datasets[0][0].x.shape[1]

print("Number of examples: ", sum(len(l) for l in list_of_datasets))
print("Shape of dataset.x: ", tuple(list_of_datasets[0][0].x.shape))

Number of examples:  1004
Shape of dataset.x:  (24, 2)


In [5]:
from collections import Counter

training_dataset_pos = []
testing_dataset_pos = []
training_dataset_neg = []
testing_dataset_neg = []

# !!! choose which one get the positive label
positive_index = 2 # just arbitrary

# relabel the datasets
for i, dataset in enumerate(list_of_datasets):
    for item in dataset:
        if i == positive_index:
            item.y = torch.tensor([1])
        else:
            item.y = torch.tensor([0])

    train_items, test_items = train_test_split(dataset, test_size=TEST_TRAIN_SPLIT)
    
    if i == positive_index:
        training_dataset_pos.extend(train_items)
        testing_dataset_pos.extend(test_items)
    else:
        training_dataset_neg.extend(train_items)
        testing_dataset_neg.extend(test_items)


class SimpleGraphDataset(InMemoryDataset):
    def __init__(self, data_list):
        super(SimpleGraphDataset, self).__init__('.', None, None, None)
        self.data, self.slices = self.collate(data_list)  # Collate all data objects

    def __len__(self):
        return len(self.data.y)  # Number of graphs in the dataset
    
    def statistics(self) -> str:
        class_counts = Counter(self.data.y.cpu().numpy())
        return " | ".join([f"{item_class}: {count}" for item_class, count in class_counts.items()]) + f" | Total: {sum(class_counts.values())}"
        

train_items = [e.to(device) for e in (training_dataset_pos + training_dataset_neg)]
test_items = [e.to(device) for e in (testing_dataset_pos + testing_dataset_neg)]

dataset = SimpleGraphDataset(train_items)
test_dataset = SimpleGraphDataset(test_items)

print("Train dataset statistics: ", dataset.statistics())
print("Test dataset statistics:  ", test_dataset.statistics())

Train dataset statistics:  1: 210 | 0: 592 | Total: 802
Test dataset statistics:   1: 53 | 0: 149 | Total: 202


/home/filip/Documents/github/Keystroke-GNN/.venv/lib/python3.12/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [6]:
# Train the model 

from torch_geometric.loader import DataLoader
# Q: How Do we choose hidden dims size ?? !!!!!!!!!!

# Define the model, loss, and optimizer
model = LetterGNN(num_node_features=dataset.num_node_features, hidden_dim=NUM_HIDDEN_DIMS, num_classes=dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

# Train loop
def train(model, data_loader):
    model.train()
    total_loss = 0
    for data in data_loader:
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.batch)  # Forward pass
        loss = criterion(output, data.y)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the model parameters
        total_loss += loss.item()
    return total_loss / len(data_loader)


# Training over epochs
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)
prev_loss = 100000
for epoch in range(1, NUM_EPOCHS):
    loss = train(model, data_loader)
    if epoch % 50 == 0 or epoch == NUM_EPOCHS-1:
        print(f"Epoch: {epoch:03d}, Loss: {loss:.4f}")
    if round(loss, 3) == 0:
        print(f"Epoch: {epoch:03d}, Loss: {loss:.4f}")
        break

/home/filip/Documents/github/Keystroke-GNN/.venv/lib/python3.12/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The data of the dataset is already cached, so any modifications to `data` will not be reflected when accessing its elements. Clearing the cache now by removing all elements in `dataset._data_list`. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Epoch: 050, Loss: 0.5335
Epoch: 100, Loss: 0.5442
Epoch: 150, Loss: 0.4688
Epoch: 200, Loss: 0.4490
Epoch: 250, Loss: 0.4215
Epoch: 300, Loss: 0.3909
Epoch: 350, Loss: 0.3639
Epoch: 400, Loss: 0.3271
Epoch: 450, Loss: 0.3238
Epoch: 500, Loss: 0.2372
Epoch: 550, Loss: 0.2299
Epoch: 600, Loss: 0.1956
Epoch: 650, Loss: 0.2090
Epoch: 700, Loss: 0.1520
Epoch: 750, Loss: 0.2356
Epoch: 800, Loss: 0.1384
Epoch: 850, Loss: 0.1341
Epoch: 900, Loss: 0.1345
Epoch: 950, Loss: 0.1503
Epoch: 999, Loss: 0.1161


In [7]:
# test 
def test(model, data_loader):
    model.eval()
    correct = 0
    for data in data_loader:
        output = model(data.x, data.edge_index, data.batch)
        pred = output.argmax(dim=1)  # Get the index of the max log-probability
        # print(f"Correct {data.y[0]} Pred {pred[0]}")
        if pred != data.y:
            print(output[0])
        correct += (pred == data.y).sum().item()
    return correct / len(data_loader.dataset)

def test_acc():
    # Test the model
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    accuracy = test(model, test_loader)
    print(f"Test Accuracy: {accuracy:.4f}")

test_acc()

tensor([ 0.2896, -0.7521], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([ 0.6797, -0.9201], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([ 0.6969, -1.1325], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([ 0.2690, -0.7592], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-0.1356, -0.2967], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([ 0.2292, -0.7843], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([ 0.2509, -0.7628], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-1.1903,  0.8157], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-3.2490,  3.6193], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-0.7150, -0.0030], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-0.9998,  0.3023], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-0.4899,  0.0346], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-0.6781,  0.3553], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-0.6087,  0.1891], device='cuda:0', grad_fn=<SelectBackward0>)
Test A

### Saving

In [8]:
MODEL_PATH = '../models/experimental.pth'
torch.save(model.state_dict(), MODEL_PATH)
print(f"Model saved to {MODEL_PATH}")

Model saved to ../models/experimental.pth


In [9]:
# Load the model (for inference)
loaded_model = LetterGNN(num_node_features=dataset.num_node_features, hidden_dim=NUM_HIDDEN_DIMS, num_classes=dataset.num_classes).to(device)
loaded_model.load_state_dict(torch.load(MODEL_PATH))
loaded_model.eval()  # Set the model to evaluation mode
print("Model loaded for inference.")

Model loaded for inference.


/tmp/ipykernel_12052/1325433200.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load(MODEL_PATH))


In [10]:
# Assuming you have a new data loader for inference data
inference_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Inference loop
with torch.no_grad():  # Disable gradient computation for inference
    for data in inference_loader:
        data = data.to(device)
        output = loaded_model(data.x, data.edge_index, data.batch)
        pred = output.argmax(dim=1)

In [11]:
# test
def test(model, data_loader):
    model.eval()
    correct = 0
    for data in data_loader:
        output = model(data.x, data.edge_index, data.batch)
        pred = output.argmax(dim=1)  # Get the index of the max log-probability
        # print(f"Correct {data.y[0]} Pred {pred[0]}")
        if pred != data.y:
            print(output[0])
        correct += (pred == data.y).sum().item()
    return correct / len(data_loader.dataset)

def test_acc():
    # Test the model
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    accuracy = test(loaded_model, test_loader)
    print(f"Test Accuracy: {accuracy:.4f}")

test_acc()

tensor([ 0.2896, -0.7521], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([ 0.6797, -0.9200], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([ 0.6969, -1.1325], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([ 0.2690, -0.7592], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-0.1356, -0.2967], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([ 0.2292, -0.7843], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([ 0.2509, -0.7628], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-1.1903,  0.8157], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-3.2490,  3.6193], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-0.7150, -0.0030], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-0.9998,  0.3023], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-0.4899,  0.0346], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-0.6781,  0.3553], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-0.6087,  0.1891], device='cuda:0', grad_fn=<SelectBackward0>)
Test A